In [1]:
from ase.io.trajectory import TrajectoryWriter as tw
from ase.io.trajectory import TrajectoryReader as tr

import os
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.inset_locator import InsetPosition

from publish_format import useLargeSize

/var/folders/s_/s1p_xb8n08x8djc3cxcpsm900000gn/T/ipykernel_1267/3934595311.py:10: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  from mpl_toolkits.axes_grid.inset_locator import InsetPosition


In [2]:
def extract_energy(filename):
    energy = []
    trajs = tr(filename)
    for traj in trajs:
        energy.append(traj.get_potential_energy())
    return np.array(energy)


def extract_forces(filename):
    forces = []
    trajs = tr(filename)
    for traj in trajs:
        forces.append(np.linalg.norm(np.array(traj.get_forces())))
    return np.array(forces)


def extract_geom(filename):
    '''
    "C Cl H H H"
    '''
    d_c_cl = []
    d_c_h1 = []
    d_c_h2 = []
    d_c_h3 = []
    theta_cl_c_h1 = []
    theta_cl_c_h2 = []
    theta_cl_c_h3 = []
    theta_h1_c_h2 = []
    theta_h1_c_h3 = []
    theta_h2_c_h3 = []
    
    trajs = tr(filename)
    for traj in trajs:
        d_c_cl.append(traj.get_distances(0, 1)[0])
        d_c_h1.append(traj.get_distances(0, 2)[0])
        d_c_h2.append(traj.get_distances(0, 3)[0])
        d_c_h3.append(traj.get_distances(0, 4)[0])

        theta_cl_c_h1.append(traj.get_angle(1,0,2))
        theta_cl_c_h2.append(traj.get_angle(1,0,3))
        theta_cl_c_h3.append(traj.get_angle(1,0,4))
        theta_h1_c_h2.append(traj.get_angle(2,0,3))
        theta_h1_c_h3.append(traj.get_angle(2,0,4))
        theta_h2_c_h3.append(traj.get_angle(3,0,4))
        
    data = pd.DataFrame(
        {r"$d_{C-Cl}$": d_c_cl, 
         r"$d_{C-H_{1}}$": d_c_h1, 
         r"$d_{C-H_{2}}$": d_c_h2, 
         r"$d_{C-H_{3}}$": d_c_h3,
         r"$\theta_{Cl-C-H_{1}}$": theta_cl_c_h1,
         r"$\theta_{Cl-C-H_{2}}$": theta_cl_c_h2,
         r"$\theta_{Cl-C-H_{3}}$": theta_cl_c_h3,
         r"$\theta_{H_{1}-C-H_{2}}$": theta_h1_c_h2,
         r"$\theta_{H_{1}-C-H_{3}}$": theta_h1_c_h3,
         r"$\theta_{H_{2}-C-H_{3}}$": theta_h2_c_h3,
         }
    )
    return data

In [3]:
# test fmax influence
data1 = extract_geom("../geom_opt/dft/ch3cl_opt_005.traj")
data2 = extract_geom("../geom_opt/dft/ch3cl_opt.traj")

In [6]:
data1.tail(1)

,$d_{C-Cl}$,$d_{C-H_{1}}$,$d_{C-H_{2}}$,$d_{C-H_{3}}$,$\theta_{Cl-C-H_{1}}$,$\theta_{Cl-C-H_{2}}$,$\theta_{Cl-C-H_{3}}$,$\theta_{H_{1}-C-H_{2}}$,$\theta_{H_{1}-C-H_{3}}$,$\theta_{H_{2}-C-H_{3}}$
10,1.797666,1.093926,1.09379,1.09339,108.356426,108.316915,108.39384,110.664127,110.605835,110.419887


In [7]:
data2.tail(1)

,$d_{C-Cl}$,$d_{C-H_{1}}$,$d_{C-H_{2}}$,$d_{C-H_{3}}$,$\theta_{Cl-C-H_{1}}$,$\theta_{Cl-C-H_{2}}$,$\theta_{Cl-C-H_{3}}$,$\theta_{H_{1}-C-H_{2}}$,$\theta_{H_{1}-C-H_{3}}$,$\theta_{H_{2}-C-H_{3}}$
14,1.796243,1.093529,1.093498,1.093585,108.482502,108.481182,108.493705,110.439478,110.432484,110.443274


In [3]:
file = {}

file["DFT"] = "../geom_opt/dft/ch3cl_opt_005.traj"
file["AMPtorch DFT (with force)"] = "../geom_opt/amptorch_dft_force_pyscf_ase/ch3cl_opt.traj"
file["AMPtorch DFT (without force)"] = "../geom_opt/amptorch_dft_noforce_pyscf_ase/ch3cl_opt.traj"
file["AMPtorch DMC"] = "../geom_opt/amptorch_dmc/ch3cl_opt.traj"

In [4]:
# show the running time for each trajectories
for item in file:
    print(item, " has length ", len(tr(file[item])))
    print(tr(file[item])[0].get_positions())

DFT  has length  11
[[6.94569073 7.23186609 7.54718282]
 [8.45195933 7.37958249 7.36984613]
 [6.61522498 8.24475411 7.63190391]
 [6.41580677 6.92533599 6.50385638]
 [6.40850209 6.74071976 8.6035065 ]]
AMPtorch DFT (with force)  has length  10
[[6.94569073 7.23186609 7.54718282]
 [8.45195933 7.37958249 7.36984613]
 [6.61522498 8.24475411 7.63190391]
 [6.41580677 6.92533599 6.50385638]
 [6.40850209 6.74071976 8.6035065 ]]
AMPtorch DFT (without force)  has length  14
[[6.94569073 7.23186609 7.54718282]
 [8.45195933 7.37958249 7.36984613]
 [6.61522498 8.24475411 7.63190391]
 [6.41580677 6.92533599 6.50385638]
 [6.40850209 6.74071976 8.6035065 ]]
AMPtorch DMC  has length  13
[[6.94569073 7.23186609 7.54718282]
 [8.45195933 7.37958249 7.36984613]
 [6.61522498 8.24475411 7.63190391]
 [6.41580677 6.92533599 6.50385638]
 [6.40850209 6.74071976 8.6035065 ]]


In [13]:
for item in file:
    geom = extract_geom(file[item])
    print(item)
    for column in geom.columns.values.tolist():
        print(column)
        for value in geom[column].values.tolist():
            print("{:.4f}".format(value))

DFT
$d_{C-Cl}$
1.5238
1.8301
1.8355
1.8295
1.8090
1.7935
1.7938
1.7966
1.7982
1.7985
1.7977
$d_{C-H_{1}}$
1.0688
1.0637
1.0803
1.0903
1.0899
1.0903
1.0919
1.0953
1.0956
1.0947
1.0939
$d_{C-H_{2}}$
1.2097
1.1201
1.1087
1.0824
1.0800
1.0925
1.0922
1.0937
1.0943
1.0942
1.0938
$d_{C-H_{3}}$
1.2828
1.1607
1.1272
1.0639
1.0788
1.0921
1.0950
1.0984
1.0969
1.0944
1.0934
$\theta_{Cl-C-H_{1}}$
102.8845
95.5093
96.3835
101.8407
105.4828
107.8885
107.8523
107.9166
108.0827
108.2664
108.3564
$\theta_{Cl-C-H_{2}}$
110.9273
104.0943
104.3403
106.1081
107.6970
108.4191
108.4484
108.3685
108.2974
108.2767
108.3169
$\theta_{Cl-C-H_{3}}$
123.1524
115.5199
115.2161
113.1695
111.7786
110.8935
110.6068
109.4784
108.8250
108.4412
108.3938
$\theta_{H_{1}-C-H_{2}}$
99.9666
105.2894
105.6057
108.7871
110.6666
111.2146
111.1660
110.9523
110.8420
110.7406
110.6641
$\theta_{H_{1}-C-H_{3}}$
99.6767
106.0786
106.7181
109.4105
109.7596
110.0920
110.0332
110.2164
110.4485
110.6096
110.6058
$\theta_{H_{2}-C-H_{3}}$
115